# Intermodulation Products
Intermodulation products are created by nonlinearities in analog hardware, represented of the form $\sum_{k} c[k-1] * x[n]^k$. The 3rd and 5th order terms are the most important because they fall in band due to the sums and differences of the frequency terms.

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

# TorchSig
from torchsig.signals.signal_types import Signal
from torchsig.utils.defaults import default_dataset
import torchsig.transforms.functional as F
from torchsig.utils.dsp import frequency_shift, multistage_polyphase_resampler

Two test signals are created: a QPSK signal and an unmodulated carrier (tone).

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 128.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["qpsk"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        noise_power_db=0.0,
        start_in_samples=0,
        seed=42,
    )
    signal = dataset.signal_generators[0]()

    # normalize, then scale data
    signal.data = F.normalize(data=signal.data, norm_order=2, flatten=False)
    signal.data = signal.data * scale

    return signal

In [ ]:
def generate_tone_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband Tone Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 128.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["tone"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        noise_power_db=0.0,
        start_in_samples=0,
        seed=42,
    )
    signal = dataset.signal_generators[0]()

    # normalize, then scale data
    signal.data = F.normalize(data=signal.data, norm_order=2, flatten=False)
    signal.data = signal.data * scale

    return signal

The QPSK and tone signals are created.

In [ ]:
# test data
N = 1024
qpsk_bb_data = generate_qpsk_signal(num_iq_samples=N, scale=1.0).data
tone_bb_data = generate_tone_signal(num_iq_samples=N, scale=1.0).data

# upsample, rescale, then frequency shift
qpsk_8x_data = multistage_polyphase_resampler(qpsk_bb_data, 8.0)
tone_8x_data = multistage_polyphase_resampler(tone_bb_data, 8.0)

qpsk_data = frequency_shift(qpsk_8x_data, 0.125, 1.0)
qpsk_data = qpsk_data / np.max(np.abs(qpsk_data))

tone1 = frequency_shift(tone_8x_data, 0.10, 1.0)
tone1 = tone1 / np.max(np.abs(tone1))
tone2 = frequency_shift(tone_8x_data, 0.15, 1.0)
tone2 = tone2 / np.max(np.abs(tone2))
two_tone_data = tone1 + tone2

freq_vec = np.arange(-1.0 / 2, 1.0 / 2, 1.0 / (N * 8))

The 3rd order intermodulation distortion model (IMD) is applied to the QPSK and tone signals. Notice how the frequency plot demonstrates an increase in the bandwidth of the QPSK signal, and results in two extra tones for the tone signal case.

In [ ]:
# third-order IMD model
coeffs = np.array([1.0, 0.0, 0.1])
qpsk_nl_data = F.intermodulation_products(data=qpsk_data, coeffs=coeffs)
two_tone_nl_data = F.intermodulation_products(data=two_tone_data, coeffs=coeffs)

Q = np.fft.fftshift(np.fft.fft(qpsk_data)) / (N * 8)
Q_nl = np.fft.fftshift(np.fft.fft(qpsk_nl_data)) / (N * 8)
T = np.fft.fftshift(np.fft.fft(two_tone_data)) / (N * 8)
T_nl = np.fft.fftshift(np.fft.fft(two_tone_nl_data)) / (N * 8)

low_tone_ind = int((0.10 + 0.5) * (1024 * 8))
high_im_ind = int((0.20 + 0.5) * (1024 * 8))
print(low_tone_ind, np.abs(T_nl[low_tone_ind]))
print(high_im_ind, np.abs(T_nl[high_im_ind]))
print(np.abs(T_nl[low_tone_ind]) / np.abs(T_nl[high_im_ind]))


fig, ax = plt.subplots(2, 1, figsize=(8, 8))
ax[0].plot(freq_vec, 10 * np.log10(np.abs(Q_nl * Q_nl)), "b")
ax[0].plot(freq_vec, 10 * np.log10(np.abs(Q * Q)), "k-")
ax[0].set_ylim([-100, -20])
ax[0].set_ylabel("Magnitude (log10)", fontsize="large")
ax[0].set_title("Third-order Nonlinearity Model: QPSK")
ax[0].legend(["Nonlinear", "Linear"], fontsize="large", loc="upper left")
ax[1].plot(freq_vec, 10 * np.log10(np.abs(T_nl * T_nl)), "b")
ax[1].plot(freq_vec, 10 * np.log10(np.abs(T * T)), "k-")
ax[1].set_ylim([-80, 5])
ax[1].set_xlabel("Frequency (Fs norm)", fontsize="large")
ax[1].set_xlabel("Frequency (Fs norm)", fontsize="large")
ax[1].set_ylabel("Magnitude (log10)", fontsize="large")
ax[1].set_title("Third-order Nonlinearity Model: Two Tones (0.1, 0.15)")
ax[1].legend(["Nonlinear", "Linear"], fontsize="large", loc="upper left");

The 5th order intermodulation product transform is applied to the QPSK signal. For QPSK, the result is similar to the 3rd order, however there is less visible contribution from the 5th order terms because they are smaller in magnitude. For the tone signal, a two additional tones are created but at reduced magnitudes.

In [ ]:
# fifth-order IMD model
coeffs = np.array([1.0, 0.0, 0.1, 0.0, 0.01])
qpsk_nl_data = F.intermodulation_products(data=qpsk_data, coeffs=coeffs)
two_tone_nl_data = F.intermodulation_products(data=two_tone_data, coeffs=coeffs)

Q = np.fft.fftshift(np.fft.fft(qpsk_data)) / (N * 8)
Q_nl = np.fft.fftshift(np.fft.fft(qpsk_nl_data)) / (N * 8)
T = np.fft.fftshift(np.fft.fft(two_tone_data)) / (N * 8)
T_nl = np.fft.fftshift(np.fft.fft(two_tone_nl_data)) / (N * 8)

fig, ax = plt.subplots(2, 1, figsize=(8, 8))
ax[0].plot(freq_vec, 10 * np.log10(np.abs(Q_nl * Q_nl)), "b")
ax[0].plot(freq_vec, 10 * np.log10(np.abs(Q * Q)), "k-")
ax[0].set_ylim([-100, -20])
ax[0].set_ylabel("Magnitude (log10)", fontsize="large")
ax[0].set_title("Fifth-order Nonlinearity Model: QPSK")
ax[0].legend(["Nonlinear", "Linear"], fontsize="large", loc="upper left")
ax[1].plot(freq_vec, 10 * np.log10(np.abs(T_nl * T_nl)), "b")
ax[1].plot(freq_vec, 10 * np.log10(np.abs(T * T)), "k-")
ax[1].set_ylim([-80, 10])
ax[1].set_xlabel("Frequency (Fs norm)", fontsize="large")
ax[0].set_ylabel("Magnitude (log10)", fontsize="large")
ax[1].set_title("Fifth-order Nonlinearity Model: Two Tones (0.1, 0.15)")
ax[1].legend(["Nonlinear", "Linear"], fontsize="large", loc="upper left");